# 准备数据

## data download

In [ ]:
import requests
import re
import os

# 创建存放XrayMassCoef_html的文件夹
XrayMassCoef_html_lists = "Data/XrayMassCoef_html_lists"
os.makedirs(XrayMassCoef_html_lists, exist_ok=True)

# 网页的URL
content_url = "https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html"
constant_url = "https://physics.nist.gov/PhysRefData/XrayMassCoef/tab1.html"
content_file_name = r"Data/X-Ray Mass Attenuation Coefficients content.html"
constant_file_name = r"Data/Element constant.html"
# 使用requests库下载网页
content_response = requests.get(content_url)
constant_response = requests.get(constant_url)

# 将网页内容保存到content_file
with open(content_file_name, "w", encoding='utf-8') as file:
    file.write(content_response.text)
with open(constant_file_name, "w", encoding='utf-8') as file:
    file.write(constant_response.text)


# 读取content_file
with open(content_file_name, "r", encoding='utf-8') as file:
    content = file.read()

# 使用正则表达式匹配
pattern = r'<TD>(.*?)</TD>\s*<TD><A href="(.*?)">(.*?)</A></TD>'
matches = re.findall(pattern, content)


# 对于每个匹配的元素，下载网页并保存为文件
for match in matches:
    symbol, href, name = match
    url = f"https://physics.nist.gov/PhysRefData/XrayMassCoef/{href}"
    response = requests.get(url)
    filename = XrayMassCoef_html_lists+f"/{symbol}_{name}_XrayMassCoef.html"
    with open(filename, "w", encoding='utf-8') as file:
        file.write(response.text)

## 整理数据格式

In [ ]:
# 获取元素常数信息

from bs4 import BeautifulSoup
import pandas as pd

# 创建BeautifulSoup对象
soup = BeautifulSoup(content, 'html.parser')

# 找到所有的<tr>标签
rows = soup.find_all('tr')

# 创建一个空的DataFrame来存储提取的信息
df = pd.DataFrame(columns=['Z', 'Symbol', 'Z/A', 'Density'])

# 遍历每一行
elementZA = {}
for row in rows:
    # 在当前行中找到所有的<td>标签
    cols = row.find_all('td')
    # 如果找到的<td>标签数量不足，跳过当前行
    if len(cols) < 6:
        continue
    # 提取需要的信息
    Z = cols[0].text.strip()
    symbol = cols[1].text.strip()
    z_a = cols[3].text.strip()
    density = cols[5].text.strip()
    # 将提取的信息添加到DataFrame中
    elementZA[Z] = (z_a, density) # 保存Z/A和Density
    df = pd.concat([df, pd.DataFrame({'Z': [Z], 'Symbol': [symbol], 'Z/A': [z_a], 'Density': [density]})], ignore_index=True)

# 打印DataFrame
print(df)

elementZA['1'] = ("0.99212","8.375E-05") # 特殊情况的特殊处理

In [ ]:
# 获取元素X-ray Mass Attenuation Coefficients信息

import os
from bs4 import BeautifulSoup
import pandas as pd

# 注意 C 元素的名称是 Carbon, 而不是 Carbon (graphite)，后者需要写复杂的逻辑处理，所以我手动处理了源html文件
# 注意，Be元素的格式提取有点问题， 执行下面代码后手动看一看

XrayMassCoef_html_lists = "Data/XrayMassCoef_html_lists"
XrayMassCoef_csv_lists = "Data/XrayMassCoef_csv_lists"
os.makedirs(XrayMassCoef_csv_lists, exist_ok=True)

# 获取XrayMassCoef_html_lists文件夹中的所有文件
files = os.listdir(XrayMassCoef_html_lists)

Zlist = []
title = ['Energy (MeV)', 'μ/ρ (cm2/g)', 'μen/ρ (cm2/g)']
# 对于每个文件，提取<PRE>标签内的内容并保存为CSV文件
for file in files:
    with open(XrayMassCoef_html_lists+f"/{file}", "r", encoding='utf-8') as datafile:
        content = datafile.read()
    soup = BeautifulSoup(content, 'html.parser')
    Z = soup.find('b').get_text().split()[3]
    pre_content = soup.find('pre').get_text()
    data = []
    for line in pre_content.split('\n'):
        if line:
            spilt_list = line.split()
            if len(spilt_list) == 4:
                data.append(spilt_list[1:])
            if len(spilt_list) <= 3:
                data.append(spilt_list)

    df = pd.DataFrame(data[4:], columns=title)
    symbol, name, _ = file.split('_')
    Zlist.append(Z)
    A = float(Z) / float(elementZA[Z][0]) 
    print(f"/{symbol}_{name}_{Z}_{A:.2f}_XrayMassCoef.csv")
    df.to_csv(XrayMassCoef_csv_lists+f"/{symbol}_{name}_{Z}_{A:.2f}_XrayMassCoef.csv", index=False)